In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
num_features = 4  # Number of input features
model = Sequential([
    Dense(64, activation='relu', input_shape=(num_features,)),  # Input layer
    Dropout(0.2),  # Dropout for regularization
    Dense(32, activation='relu'),  # Hidden layer
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,433 (9.50 KB)

 Trainable params: 2,433 (9.50 KB)

 Non-trainable params: 0 (0.00 B)

**Sensor	      Normal Range	Fault Range**
**Temperature**	   30–60°C	      60–100°C
**Vibration**	    0.2–0.7 g	       > 0.7 g
**Current**	        8–12 A	   < 8 A or > 15 A
**Voltage	**      220–240 V	   < 210 V or > 250 V
**RPM**	         1450–1550	   < 1400 or > 1600

In [ ]:
import numpy as np
import pandas as pd

n = 1000
fault_prob = 0.4  # 40% faulty records

# Normal Ranges
voltage = np.random.uniform(210, 230, n)
current = np.random.uniform(0.35, 0.65, n)
vibration = np.random.uniform(-0.5, 0.5, n)
rpm = np.random.uniform(2800, 3200, n)

# Fault Label
fault = np.random.choice([0, 1], size=n, p=[1 - fault_prob, fault_prob])

# Inject faults
for i in range(n):
    if fault[i] == 1:
        fault_type = np.random.choice(["voltage", "current", "vibration", "rpm"])
        if fault_type == "voltage":
            voltage[i] = np.random.choice([np.random.uniform(180, 209), np.random.uniform(241, 260)])
        elif fault_type == "current":
            current[i] = np.random.choice([np.random.uniform(0.1, 0.3), np.random.uniform(1.0, 1.5)])
        elif fault_type == "vibration":
            vibration[i] = np.random.choice([np.random.uniform(-1.0, -0.6), np.random.uniform(0.6, 1.0)])
        elif fault_type == "rpm":
            rpm[i] = np.random.choice([np.random.uniform(2400, 2700), np.random.uniform(3300, 3600)])

# Combine into DataFrame
data = pd.DataFrame({
    'Voltage (V)': voltage,
    'Current (A)': current,
    'Vibration (g)': vibration,
    'RPM': rpm,
    'Fault': fault
})

# Save to CSV
# data.to_csv("induction_motor_1000_data.csv", index=True)
print(data)


     Voltage (V)  Current (A)  Vibration (g)          RPM  Fault
0     227.897155     0.384271       0.944523  3001.761360      1
1     214.950761     0.208256       0.085388  2878.397181      1
2     220.142462     0.375494       0.343507  2943.378577      0
3     216.298951     0.592129       0.032239  2871.287967      0
4     217.816690     0.538084       0.733954  2830.674723      1
..           ...          ...            ...          ...    ...
995   243.269622     0.386331      -0.228678  3166.430914      1
996   211.706759     0.393600       0.288340  3077.839108      0
997   224.905537     0.494100       0.390701  2962.371587      0
998   245.527869     0.386957      -0.053268  3151.697260      1
999   225.619381     1.438819       0.250063  3096.917286      1

[1000 rows x 5 columns]


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

x = data.drop("Fault", axis=1)
y = data["Fault"]


scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


# Initialize a label encoder
encoder = LabelEncoder()

# Encode the labels
y_encoded = encoder.fit_transform(y)

# Split the dataset (80% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_encoded, test_size=0.3, random_state=42)

# Further split training set into training and validation (80% train, 30% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Print the shapes to verify
print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}, Test shape: {X_test.shape}")

Train shape: (490, 4), Validation shape: (210, 4), Test shape: (300, 4)


In [ ]:
import joblib

joblib.dump(scaler, 'motor_scaler.pkl')

['motor_scaler.pkl']

In [ ]:
# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=60,  # Number of training epochs
                    batch_size=32,  # Number of samples per training step
                    verbose=1)  # Show training progress


Epoch 1/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 148ms/step - accuracy: 0.4043 - loss: 0.6746 - val_accuracy: 0.3857 - val_loss: 0.6693
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4911 - loss: 0.6434 - val_accuracy: 0.4333 - val_loss: 0.6534
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5358 - loss: 0.6423 - val_accuracy: 0.5952 - val_loss: 0.6371
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6820 - loss: 0.6117 - val_accuracy: 0.6905 - val_loss: 0.6183
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6881 - loss: 0.6142 - val_accuracy: 0.8762 - val_loss: 0.5934
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7960 - loss: 0.5705 - val_accuracy: 0.9143 - val_loss: 0.5663
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8058 - loss: 0.5519 - val_accuracy: 0.9381 - val_loss: 0.5323
Epoch 8/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8753 - loss: 0.5112 - val_accuracy: 0.9333 - val_los

In [ ]:
# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 96.67%


In [ ]:

# Test input with actual sensor reading format:
# [Voltage (V), Current (A), Vibration (g), RPM]
test_input = [[220.0, 0.45, -0.35, 3060.0]]  # Real motor example

# Ensure the scaler matches the new feature layout
test_input_scaled = scaler.transform(test_input)

# Predict using your trained model
predicted_label = (model.predict(test_input_scaled) > 0.5).astype(int)

# Output result
if predicted_label[0][0] == 1:
    print("Result: Needs Maintenance")
else:
    print("Result: Normal")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Result: Normal


In [ ]:
test_inputs = np.array([
    [220.0, 0.45, -0.35, 3060.0],  # Normal
    [205.0, 0.25, 0.65, 2700.0],   # Fault: low voltage, high vibration, low RPM
    [230.0, 0.60, 0.30, 3150.0],   # Normal
    [250.0, 1.20, 0.75, 3400.0],   # Fault: high voltage, high current, high RPM
    [210.0, 0.40, -0.20, 2900.0],  # Normal
    [198.0, 1.10, -0.60, 2680.0],  # Fault: low voltage, high current, low RPM
    [215.0, 0.50, -0.45, 3080.0],  # Normal
    [245.0, 1.25, 0.85, 3500.0],   # Fault: all abnormal
    [222.0, 0.38, -0.10, 3000.0],  # Normal
    [218.0, 0.90, 0.55, 3300.0],   # Fault: borderline high current & vibration
], dtype=np.float32)

# Apply the same scaler used during training
test_inputs_scaled = scaler.transform(test_inputs)

# Run predictions
predictions = (model.predict(test_inputs_scaled) > 0.5).astype(int)

# Display results
print("Test Results:\n")
for i, (input_data, prediction) in enumerate(zip(test_inputs, predictions)):
    label = "Needs Maintenance" if prediction[0] == 1 else "Normal"
    print(f"Sample {i+1}: {input_data} => Prediction: {label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Test Results:

Sample 1: [ 2.20e+02  4.50e-01 -3.50e-01  3.06e+03] => Prediction: Normal
Sample 2: [2.05e+02 2.50e-01 6.50e-01 2.70e+03] => Prediction: Needs Maintenance
Sample 3: [2.30e+02 6.00e-01 3.00e-01 3.15e+03] => Prediction: Normal
Sample 4: [2.5e+02 1.2e+00 7.5e-01 3.4e+03] => Prediction: Needs Maintenance
Sample 5: [ 2.1e+02  4.0e-01 -2.0e-01  2.9e+03] => Prediction: Normal
Sample 6: [ 1.98e+02  1.10e+00 -6.00e-01  2.68e+03] => Prediction: Needs Maintenance
Sample 7: [ 2.15e+02  5.00e-01 -4.50e-01  3.08e+03] => Prediction: Normal
Sample 8: [2.45e+02 1.25e+00 8.50e-01 3.50e+03] => Prediction: Needs Maintenance
Sample 9: [ 2.22e+02  3.80e-01 -1.00e-01  3.00e+03] => Prediction: Normal
Sample 10: [2.18e+02 9.00e-01 5.50e-01 3.30e+03] => Prediction: Needs Maintenance


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
model.save("motor_model.h5")
print("Model saved as 'motor_model.h5'")

Model saved as 'motor_model.h5'


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model_reloaded = load_model("motor_model.h5")
print("Model loaded successfully")

Model loaded successfully


In [ ]:
test_input = [[50.0, 0.2, 10, 220, 1450]]
test_input_scaled = scaler.transform(test_input)

# Predict using the reloaded model
predicted_label = (model_reloaded.predict(test_input_scaled) > 0.5).astype(int)

if predicted_label[0][0] == 1:
    print("Result: Needs Maintenance")
else:
    print("Result: Normal")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
Result: Normal


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import numpy as np

# 10 input samples: [Temperature, Vibration, Current, Voltage, RPM]
test_inputs = np.array([
    [40.0, 0.4, 10.0, 230.0, 1500.0],  # Normal
    [75.0, 1.2, 16.0, 210.0, 1350.0],  # Fault
    [45.0, 0.6, 11.0, 235.0, 1480.0],  # Normal
    [85.0, 0.5, 10.0, 250.0, 1500.0],  # Fault
    [38.0, 0.3, 9.0, 225.0, 1490.0],   # Normal
    [65.0, 0.8, 17.0, 240.0, 1450.0],  # Fault
    [42.0, 0.5, 10.0, 230.0, 1550.0],  # Normal
    [60.0, 1.1, 13.0, 260.0, 1600.0],  # Fault
    [30.0, 0.2, 8.0, 220.0, 1500.0],   # Normal
    [55.0, 1.3, 14.0, 215.0, 1390.0],  # Fault
])

# Scale the inputs using your original scaler
test_inputs_scaled = scaler.transform(test_inputs)

# Run predictions
predictions = (model.predict(test_inputs_scaled) > 0.5).astype(int)

# Print results
print("Results:\n")
for i, (raw_input, pred) in enumerate(zip(test_inputs, predictions)):
    status = "Needs Maintenance" if pred[0] == 1 else "Normal"
    print(f"Sample {i+1}: {raw_input} => Prediction: {status}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 5 features, but StandardScaler is expecting 4 features as input.

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,499 (9.77 KB)

 Trainable params: 2,497 (9.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)